# Limpeza dos dados
Nesse notebook vamos preparar os dados que utilizaremos ao longo do desenvolvimento do projeto. Os datasets aqui utilizados são importados do portal de dados abertos da UFRN (http://dados.ufrn.br/).

Para esse estudo, analisaremos apenas os dados do curso de Ciência e Tecnologia da UFRN, mais espeficamente a disciplina de Cálculo 2, onde tentaremos prever as possiveis reprovações dos alunos se baseando em seu histórico e na nota da primeira unidade na disciplina.

In [1]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# selecione o ano_periodo para análise
ano_periodo = '20152'

In [3]:
matriculas = pd.read_csv(f"data/matricula-de-{ano_periodo}.csv", sep=";")

In [4]:
matriculas.shape

(657934, 11)

In [5]:
matriculas.columns

Index(['id_turma', 'discente', 'id_curso', 'unidade', 'nota', 'reposicao',
       'faltas_unidade', 'media_final', 'numero_total_faltas', 'descricao',
       'Unnamed: 10'],
      dtype='object')

## Seleção dos dados
Vamos selecionar apenas os dados que nos interessam, assim, iniciaremos pegando apenas os dados do curso de CET e depois limitaremos as displinas que nos interessam.

In [6]:
id_cet = 10320810
alunos_cet = matriculas[matriculas.id_curso == id_cet]

In [7]:
alunos_cet.shape

(64973, 11)

### Pegando as turmas
Vamos pegar as turmas dos componentes que nos interessam.

In [8]:
turmas = pd.read_csv(f"data/turmas-{ano_periodo}.csv", sep=";")

In [9]:
turmas.columns

Index(['id_turma', 'codigo_turma', 'id_docente_interno', 'id_docente_externo',
       'observacao', 'id_componente_curricular', 'nivel_ensino',
       'campus_turma', 'local', 'ano', 'periodo', 'data_inicio', 'data_fim',
       'descricao_horario', 'total_solicitacoes', 'capacidade_aluno', 'tipo',
       'distancia', 'data_consolidacao', 'agrupadora', 'id_turma_agrupadora',
       'qtd_aulas_lancadas', 'situacao_turma', 'convenio',
       'modalidade_participantes', 'Unnamed: 25'],
      dtype='object')

In [10]:
turmas['ano_periodo'] = turmas[['ano', 'periodo']].apply(lambda x: x['ano']*10 + x['periodo'], axis=1)

In [12]:
turmas.head()

,id_turma,codigo_turma,id_docente_interno,id_docente_externo,observacao,id_componente_curricular,nivel_ensino,campus_turma,local,ano,...,distancia,data_consolidacao,agrupadora,id_turma_agrupadora,qtd_aulas_lancadas,situacao_turma,convenio,modalidade_participantes,Unnamed: 25,ano_periodo
0,57560974,01,5755580.0,NaN,NaN,55731,LATO SENSU,NaN,Caicó,2015,...,f,2016-02-29 16:58:56.653,f,NaN,20.0,CONSOLIDADA,NaN,Presencial,NaN,20152
1,57560975,01,5756402.0,NaN,NaN,55770,LATO SENSU,NaN,Caicó,2015,...,f,2016-09-26 00:28:07.764,f,NaN,20.0,CONSOLIDADA,NaN,Presencial,NaN,20152
2,57560976,01,22193.0,NaN,NaN,55743,LATO SENSU,NaN,Caicó,2015,...,f,2016-01-07 16:54:55.815,f,NaN,20.0,CONSOLIDADA,NaN,Presencial,NaN,20152
3,57560977,01,5755386.0,NaN,NaN,55734,LATO SENSU,NaN,Caicó,2015,...,f,2016-01-29 17:20:44.777,f,NaN,20.0,CONSOLIDADA,NaN,Presencial,NaN,20152
4,57560982,01,5755973.0,NaN,NaN,55735,LATO SENSU,NaN,Currais Novos,2015,...,f,2016-01-27 16:44:08.695,f,NaN,30.0,CONSOLIDADA,NaN,Presencial,NaN,20152


In [13]:
columns_sel = ['id_turma', 'id_componente_curricular', 'id_docente_interno',
       'ano_periodo', 'situacao_turma']
turmas = turmas[columns_sel]

In [14]:
turmas.ano_periodo.unique()

array([20152])

In [15]:
id_turmas_cet = matriculas.id_turma.unique()

In [16]:
turmas_cet = turmas[turmas.id_turma.isin(id_turmas_cet)]

In [17]:
turmas_cet.shape

(14021, 5)

In [19]:
# turmas_cet[turmas_cet.codigo.str.startswith("ECT")].codigo.unique()

### Relacionando com os componentes curriculares

In [21]:
comp_curriculares = pd.read_csv("data/componentes-curriculares-presenciais-ok.csv", sep=",")

In [23]:
cc_ect = comp_curriculares[comp_curriculares.unidade_responsavel == 'ESCOLA DE CIÊNCIAS E TECNOLOGIA']

In [27]:
disciplinas_desejadas_nome = ['PRE-CÁLCULO', 'VETORES E GEOMETRIA ANALÍTICA', 'CÁLCULO I', 'CÁLCULO II']

disciplinas_desejadas_codigo = ['ECT2101', 'ECT2102', 'ECT2103', 'ECT2201']

id_componentes_desejados = ['57586', 
                            '48582', '57587', '2050801', # cálculo 1
                            '48584', '57588', '2051052', # cálculo 2
                            '2051005'] # VGA

In [30]:
cc_ect

,Unnamed: 0,id_componente,tipo_componente,codigo,nivel,nome,unidade_responsavel,ch_teorico,ch_pratico,ch_estagio,...,ementa,bibliografia,objetivos,conteudo,competencias_habilidades,referencias,ano_programa,periodo_programa,modalidade,curso_componente
12110,12110,41879,MÓDULO,ECT1402,G,MECÂNICA DOS SÓLIDOS,ESCOLA DE CIÊNCIAS E TECNOLOGIA,82,8,0,...,"Estática: Sistemas de Forças em 2D: Força, Mom...",NaN,Apresentar os conceitos fundamentais da mecâni...,1 - SISTEMAS DE FORÇAS1.1 - Conceitos básicos;...,Calcular reações de apoio em estruturas isostá...,NaN,2012.0,1.0,Presencial,NaN
12111,12111,41880,MÓDULO,ECT1403,G,MECÂNICA DOS FLUIDOS,ESCOLA DE CIÊNCIAS E TECNOLOGIA,54,6,0,...,Definic&#807;a&#771;o de Fluido. Hipo&#769;tes...,NaN,São objetivos do curso apresentar e aplicar os...,1. Introdução e Conceitos Fundamentais ...,"Ao final do curso, os alunos deverão demonstra...",NaN,2012.0,1.0,Presencial,NaN
12112,12112,41881,MÓDULO,ECT1404,G,ELETRICIDADE APLICADA,ESCOLA DE CIÊNCIAS E TECNOLOGIA,78,12,0,...,Elementos e leis fundamentais de circuitos. Ci...,NaN,Tornar o aluno apto a aplicar os conhecimentos...,1. Conceitos e Grandezas Elétricas Básicas (Co...,As habilidades e competências referentes a ess...,NaN,2012.0,1.0,Presencial,NaN
12113,12113,41882,MÓDULO,ECT1405,G,MODELAGEM INTEGRADA,ESCOLA DE CIÊNCIAS E TECNOLOGIA,54,6,0,...,Modelagem de Dados Nume&#769;ricos. Intervalos...,NaN,Capacitar o estudante a modelar e simular sist...,1.\tIntrodução a Modelagem e Simulações1.1.\tC...,Desenvolver no aluno a habilidade de vizualiza...,NaN,2015.0,1.0,Presencial,NaN
12114,12114,41883,MÓDULO,ECT1406,G,EXPRESSÃO GRÁFICA,ESCOLA DE CIÊNCIAS E TECNOLOGIA,20,10,0,...,Introduc&#807;a&#771;o ao Desenho Te&#769;cnic...,NaN,Os objetivos do componente será definido breve...,"1. Introdução ao Desenho Técnico.(Normas ABNT,...",As competências e habilidades do aluno ao térm...,NaN,2011.0,1.0,Presencial,NaN
12473,12473,42261,MÓDULO,ECT1503,G,MATEMÁTICA FINANCEIRA,ESCOLA DE CIÊNCIAS E TECNOLOGIA,60,0,0,...,Equivale&#770;ncia de capitais no tempo. Equiv...,NaN,O objetivo do componente é introduzir o estuda...,1.\tCálculo financeiro básico1.1.\tIntrodução ...,Busca-se desenvolver com o estudante suas comp...,NaN,2010.0,2.0,Presencial,NaN
13240,13240,43343,MÓDULO,ECT1910,G,NEUROBIOLOGIA CELULAR,ESCOLA DE CIÊNCIAS E TECNOLOGIA,52,8,0,...,"Organização estrutural e molecular da célula, ...",NaN,Fornecer conhecimentos aprofundados sobre aspe...,Definição de célula e suas características uni...,Desenvolver o hábito do estudo e interpretação...,NaN,2011.0,1.0,Presencial,CIÊNCIAS E TECNOLOGIA
13241,13241,43344,MÓDULO,ECT1911,G,NEUROFISIOLOGIA,ESCOLA DE CIÊNCIAS E TECNOLOGIA,90,0,0,...,Introduzir as bases fundamentais do funcioname...,NaN,"Introduzir o aluno ao mundo da neurociência, f...",Conceitos de neurociênciaIntrodução à filogeni...,Desenvolver o hábito do estudo e interpretação...,NaN,2011.0,1.0,Presencial,CIÊNCIAS E TECNOLOGIA
13242,13242,43345,MÓDULO,ECT1912,G,SEMINÁRIOS EM NEUROCIÊNCIA,ESCOLA DE CIÊNCIAS E TECNOLOGIA,30,0,0,...,Apresentar os descobertas recentes em neurociê...,NaN,Introduzir o aluno ao cotidiano do fazer neuro...,Linhas de pesquisa desenvolvidas pelos profess...,Desenvolver o hábito do estudo e interpretação...,NaN,2011.0,1.0,Presencial,CIÊNCIAS E TECNOLOGIA
13243,13243,43346,MÓDULO,ECT1913,G,BIOFÍSICA NEURAL,ESCOLA DE CIÊNCIAS E TECNOLOGIA,30,0,0,...,(MAT0311 OU MAT0340 OU MAT0228 OU MAT0318 OU M...,NaN,Conhecer e compreender os fenômenos físicos e ...,Movimento iônico em membranas excitáveis (mode...,Desenvolver o hábito do estudo e interpretação...,NaN,2011.0,1.0,Presencial,CIÊNCIAS E TECNOLOGIA


In [29]:
cc_ect[cc_ect.codigo.isin(disciplinas_desejadas_codigo)]

,Unnamed: 0,id_componente,tipo_componente,codigo,nivel,nome,unidade_responsavel,ch_teorico,ch_pratico,ch_estagio,...,ementa,bibliografia,objetivos,conteudo,competencias_habilidades,referencias,ano_programa,periodo_programa,modalidade,curso_componente
22803,22803,57586,MÓDULO,ECT2101,G,PRÉ-CÁLCULO,ESCOLA DE CIÊNCIAS E TECNOLOGIA,60,0,0,...,Números reais. Operações aritméticas. Números ...,NaN,&#8722; Revisar e proporcionar maior compreens...,1. Números reais1.1 Representação fracionária ...,Desenvolver no aluno a habilidade de entender ...,NaN,2014.0,2.0,Presencial,CIÊNCIAS E TECNOLOGIA
22804,22804,57587,MÓDULO,ECT2103,G,CÁLCULO I,ESCOLA DE CIÊNCIAS E TECNOLOGIA,60,0,0,...,Limites e continuidade. Derivada. Aplicações d...,NaN,- Introduzir e capacitar o aluno a entender e ...,1. Limites1.1 Definição. Limites laterais.1.2 ...,Capacitar o aluno a entender e utilizar adequa...,NaN,2016.0,2.0,Presencial,CIÊNCIAS E TECNOLOGIA
22805,22805,57588,MÓDULO,ECT2201,G,CÁLCULO II,ESCOLA DE CIÊNCIAS E TECNOLOGIA,60,0,0,...,Integral definida. Aplicações da integral. Fun...,NaN,- Introduzir o conceito de integral definida b...,1. Integral Definida1.1 Definição e interpreta...,Possibilitar que o aluno tenha domínio dos con...,NaN,2014.0,2.0,Presencial,CIÊNCIAS E TECNOLOGIA
34962,34962,2051005,MÓDULO,ECT2102,G,VETORES E GEOMETRIA ANALÍTICA,ESCOLA DE CIÊNCIAS E TECNOLOGIA,60,0,0,...,Matrizes e Determinantes. Sistemas Lineares. V...,NaN,&#8722; Apresentar métodos de resolução de sis...,1. Matrizes e Sistemas Lineares1.1 Matrizes1.1...,Desenvolver no aluno a habilidade de identific...,NaN,2014.0,2.0,Presencial,CIÊNCIAS E TECNOLOGIA


In [ ]:
turmas_cet = turmas_cet[turmas_cet.codigo.isin(disciplinas_desejadas_codigo)]

In [ ]:
turmas_cet.shape

In [ ]:
turmas_cet.situacao_turma.value_counts()

### Pegando apenas as matriculas nas turmas que interessam
Hora de filtrar apenas as matriculas nas turmas dos componentes curriculares que utilizaremos.

In [ ]:
turmas_ect_selecionadas = turmas_cet.id_turma.unique()

In [ ]:
matriculas_selecionadas =  matriculas[matriculas.id_turma.isin(turmas_ect_selecionadas)]

In [ ]:
matriculas_selecionadas.columns

Removendo os status indesejados

In [ ]:
matriculas_selecionadas.descricao.value_counts()

In [ ]:
status_indesejados = ['INDEFERIDO', 'EXCLUIDA', 'TRANCADO', 'DESISTENCIA']

In [ ]:
matriculas_selecionadas = matriculas_selecionadas[~matriculas_selecionadas.descricao.isin(status_indesejados)]

## Salvando os datasets

In [ ]:
matriculas_selecionadas.to_csv(f"data/matriculas_pos_limpeza_{ano_periodo}.csv", sep=";")

In [ ]:
turmas_cet.to_csv(f"data/turmas_pos_limpeza_{ano_periodo}.csv", sep=";")